### Pipeline OCR (Tesseract + OpenCV)28.

In [2]:
import cv2
import pytesseract
import os

# -------- Config Tesseract --------
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
os.environ["TESSDATA_PREFIX"] = r"C:\Program Files\Tesseract-OCR\tessdata"

# -------- Charger --------
img = cv2.imread("imagefr.jpg")

# -------- EDSR x4 --------
sr = cv2.dnn_superres.DnnSuperResImpl_create()
sr.readModel("EDSR_x4.pb")
sr.setModel("edsr", 4)

edsr = sr.upsample(img)

# -------- OCR --------
config = "--psm 3 --oem 3"

text = pytesseract.image_to_string(
    edsr,
    lang="fra+eng",
    config=config
)

print("\n===== OCR EDSR x4 =====")
print(text)

# -------- Affichage --------
cv2.imshow("EDSR x4", edsr)

cv2.waitKey(0)
cv2.destroyAllWindows()



===== OCR EDSR x4 =====
FRANÇOIS RABELAIS
LE CINQUIÈME LIVRE

« …Je prouverai par des arguments non imper-
tinents et des raisons irréfutables, à la barbe de
je ne sais quels faiseurs de centons, borteleurs
de matiéres cent et cent fois ressassées, rapetas-
seurs de vieilles ferrailles latines, revendeurs de
vieux mots latins tout moisis et incertains, que
notre langue vulgaire n'est pas aussi vile, aussi
inepte, aussi indigente et méprisable qu'ils l'es-
timent. (...) C’est pourquoi, buveurs, je vous
avertis en temps opportun, faites de mes livres
bonne provision aussitét que vous les trouve-
rez dans les boutiques des libraires, et il vous
faudra non seulement les écosser mais encore
les dévorer comme un opiacé pour le coeur et
les incorporer en vous-mêmes ; c'est alors que
vous découvrirez le bien qu'ils réservent à tous
les nobles écosseurs de fèves, »

FRANCO RABELAIS

Translation, préface es mores par Guy Demerson

Acudoé Detain, Kadeber. bau grant 1945
© ADAGE. Pate 1997
Archiv